Import libraries

In [4]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import  precision_score, recall_score, f1_score, average_precision_score, precision_recall_curve,accuracy_score, roc_auc_score
import joblib

Training dataset in deep kernel once class classification model

In [5]:
training_dataset = pd.read_csv('Training_heart.csv')

smote = SMOTE()

# features that is not the target label
X = training_dataset.drop(' HEALTHY HEART', axis=1)
y = training_dataset.drop([' PULSE', ' SpO2'], axis=1)

y = np.reshape(y, (-1, 1))

X_train_resampled, y_train_resampled = smote.fit_resample(X, y)

# partitioning data for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle=True, random_state=42)

X_train, y_train = smote.fit_resample(X_train, y_train)

positive_values = np.count_nonzero(y_test ==  1)

# Create and train the One-Class SVM model
model = SVC(kernel='rbf', C=0.1, gamma = 1.00) 

model.fit(X_train, y_train )

svm_pred_train = model.predict(X_train)

# Gradient Boosting
gb_model = GradientBoostingClassifier(n_estimators=500,  learning_rate=0.1, random_state=42, validation_fraction=0.16, n_iter_no_change = 5)
gb_model.fit(X_train, y_train)

gb_pred_test = gb_model.predict(X_test)

positive_percentage = (y_test).sum() / len(gb_pred_test)

# Compute evaluation metrics
precision = precision_score(y_test, gb_pred_test)
precision = precision_score(y_test, gb_pred_test)
recall = recall_score(y_test, gb_pred_test)
f1 = f1_score(y_test, gb_pred_test)
roc_auc = roc_auc_score(y_test, gb_pred_test)
auprc = average_precision_score(y_test, gb_pred_test)
accuracy = accuracy_score(y_test, gb_pred_test)


# Print the evaluation scores
'''
precision = precision_score(y_val, y_test_pred)
recall = recall_score(y_val, y_test_pred)'''

print ("Positive percentage:", positive_percentage)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)  
print("Accuracy ", accuracy)
print("ROC AUC", roc_auc)
print("AUPRC:", auprc)

filename = 'Heart_attack_monitor_SVM_GB.sav'
joblib.dump(gb_model, filename)

Positive percentage: 0.7725875845113707
Precision: 1.0
Recall: 1.0
F1-score: 1.0
Accuracy  1.0
ROC AUC 1.0
AUPRC: 1.0


['Heart_attack_monitor_SVM_GB.sav']

Testing the SVM modle performance 

In [6]:
# Load a model
dkrlvoc = joblib.load('Heart_attack_monitor_SVM_GB.sav')

val_dataset = pd.read_csv('Validate_heart.csv')
x_val = val_dataset.drop('Normal heart', axis=1)

y_val = val_dataset['Normal heart']  
#y_val_features = np.reshape(y_val, (-1, 1))

y_pred = dkrlvoc.predict(x_val)
y_pred = np.where(y_pred == -1, 0, 1)

positive_percentage = (y_val).sum() / len(y_pred)
precision, recall, thresholds = precision_recall_curve(y_val, y_pred)


# Compute evaluation metrics
#f1 = f1_score(y_val_features, y_pred)

roc_auc = roc_auc_score(y_val, y_pred)

# Print the evaluation scores

outlier_count = (y_val == -1).sum()

print ("Positive percentage:", positive_percentage)
print("Outlier count:", outlier_count)
print("Precision:", precision)
print("Recall:", recall)
print("Threshold:", thresholds)
print("ROC AUC", roc_auc)
#print("F1-score:", f1)  
#print("Accuracy ", scores.mean())

# Calculate AUPRC
auprc = average_precision_score(y_val, y_pred)
print("AUPRC:", auprc)

FileNotFoundError: [Errno 2] No such file or directory: 'Validate_heart.csv'